In [1]:
pip install opencv-python
pip install deepface
pip install tf-keras

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
from deepface import DeepFace

# Configuration constants
CASCADE_PATH = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
SCALE_FACTOR = 1.1
MIN_NEIGHBORS = 5
MIN_FACE_SIZE = (30, 30)

def initialize_video_capture():
    """Initialize the video capture device."""
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise RuntimeError("Error: Could not open video capture.")
    return cap

def detect_faces(face_cascade, frame):
    """Detect faces in the given frame."""
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray_frame, 
        scaleFactor=SCALE_FACTOR, 
        minNeighbors=MIN_NEIGHBORS, 
        minSize=MIN_FACE_SIZE
    )
    return faces

def analyze_emotions(face_roi):
    """Analyze emotions using DeepFace."""
    try:
        result = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)
        return result[0]['dominant_emotion']
    except Exception as e:
        print(f"Emotion analysis error: {e}")
        return "Unknown"

def main():
    # Load face cascade classifier
    face_cascade = cv2.CascadeClassifier(CASCADE_PATH)

    # Start capturing video
    cap = initialize_video_capture()

    try:
        while True:
            # Capture frame-by-frame
            ret, frame = cap.read()
            if not ret:
                print("Error: Unable to read frame.")
                break

            # Detect faces
            faces = detect_faces(face_cascade, frame)

            for (x, y, w, h) in faces:
                # Extract and validate the face ROI
                face_roi = frame[y:y + h, x:x + w]
                if face_roi.size == 0:
                    continue

                # Analyze emotions
                emotion = analyze_emotions(face_roi)

                # Draw rectangle and label around the face
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

            # Display the resulting frame
            cv2.imshow('Real-time Emotion Detection', frame)

            # Press 'q' to exit
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        # Release the capture and close all windows
        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


2025-01-10 15:45:14.096097: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


25-01-10 15:45:28 - facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /Users/rgopalam/.deepface/weights/facial_expression_model_weights.h5
100%|██████████| 5.98M/5.98M [00:00<00:00, 18.2MB/s]


: 

In [ ]:
import cv2
from deepface import DeepFace
import numpy as np
from scipy.spatial.distance import cosine

# Configuration constants
CASCADE_PATH = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
SCALE_FACTOR = 1.1
MIN_NEIGHBORS = 5
MIN_FACE_SIZE = (30, 30)
EMBEDDING_THRESHOLD = 0.15  # Distance threshold for face matching
PROFILE_EXPIRATION_TIME = 10  # Time in seconds

# Global variable to store profiles
profiles = {}

def load_known_faces(image_paths):
    """
    Load known faces and generate embeddings.
    :param image_paths: Dictionary of {name: image_path} pairs.
    :return: Dictionary of {name: embedding} pairs.
    """
    known_faces = {}
    for name, image_path in image_paths.items():
        try:
            # Load the image and generate an embedding
            face_image = cv2.imread(image_path)
            embedding = DeepFace.represent(face_image, model_name='Facenet512', enforce_detection=False)
            known_faces[name] = np.array(embedding[0]['embedding'])
        except Exception as e:
            print(f"Error loading face for {name}: {e}")
    return known_faces


def initialize_video_capture():
    """Initialize the video capture device."""
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise RuntimeError("Error: Could not open video capture.")
    return cap

def match_known_face(face_embedding, known_faces, threshold=0.4):
    """
    Match a detected face embedding to a known face.
    :param face_embedding: The embedding of the detected face.
    :param known_faces: Dictionary of {name: embedding} pairs.
    :param threshold: Distance threshold for a match.
    :return: Name of the matched face or None if no match is found.
    """
    for name, known_embedding in known_faces.items():
        distance = cosine(known_embedding, face_embedding)
        if distance < threshold:
            return name
    return None


def detect_faces(face_cascade, frame):
    """Detect faces in the given frame."""
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray_frame, 
        scaleFactor=SCALE_FACTOR, 
        minNeighbors=MIN_NEIGHBORS, 
        minSize=MIN_FACE_SIZE
    )
    return faces

def get_face_embedding(face_roi):
    """Generate an embedding for the given face ROI."""
    try:
        embedding = DeepFace.represent(face_roi, model_name='Facenet512', enforce_detection=False)
        return np.array(embedding[0]['embedding'])
    except Exception as e:
        print(f"Embedding error: {e}")
        return None

def analyze_emotions(face_roi):
    """Analyze emotions using DeepFace."""
    try:
        result = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)
        return result[0]['dominant_emotion']
    except Exception as e:
        print(f"Emotion analysis error: {e}")
        return "Unknown"

def match_profile(embedding):
    """Match the given embedding to an existing profile."""
    for profile_id, profile_data in profiles.items():
        profile_embedding = profile_data['embedding']
        distance = cosine(profile_embedding, embedding)
        if distance < EMBEDDING_THRESHOLD:
            return profile_id
    return None

def main():
    # Load face cascade classifier
    face_cascade = cv2.CascadeClassifier(CASCADE_PATH)

    # Start capturing video
    cap = initialize_video_capture()

    frame_skip = 20  # Perform analysis every 5 frames
    frame_counter = 0
    persistence_time = 15  # Number of frames to persist a bounding box
    active_faces = {}  # Store active bounding boxes and their persistence counters

    try:
        while True:
            # Capture frame-by-frame
            ret, frame = cap.read()
            if not ret:
                print("Error: Unable to read frame.")
                break

            # Increment frame counter
            frame_counter += 1

            if frame_counter % frame_skip == 0:
                # Perform face detection and analysis every `frame_skip` frames
                faces = detect_faces(face_cascade, frame)

                for (x, y, w, h) in faces:
                    # Extract and validate the face ROI
                    face_roi = frame[y:y + h, x:x + w]
                    if face_roi.size == 0:
                        continue

                    # Get face embedding
                    embedding = get_face_embedding(face_roi)
                    if embedding is None:
                        continue

                    # Match or create profile
                    profile_id = match_profile(embedding)
                    if profile_id is None:
                        # Create a new profile
                        profile_id = len(profiles) + 1
                        profiles[profile_id] = {
                            'embedding': embedding,
                            'emotions': [],
                            'last_seen': cv2.getTickCount() / cv2.getTickFrequency()
                        }

                    # Analyze emotions
                    emotion = analyze_emotions(face_roi)
                    profiles[profile_id]['emotions'].append(emotion)
                    profiles[profile_id]['last_seen'] = cv2.getTickCount() / cv2.getTickFrequency()

                    # Update active_faces with new detection
                    active_faces[profile_id] = {
                        'box': (x, y, w, h),
                        'emotion': emotion,
                        'counter': persistence_time  # Reset the persistence counter
                    }

            # Decrement counters for inactive faces
            for profile_id in list(active_faces.keys()):
                active_faces[profile_id]['counter'] -= 1
                if active_faces[profile_id]['counter'] <= 0:
                    del active_faces[profile_id]  # Remove expired bounding boxes

            # Draw all active bounding boxes
            for profile_id, data in active_faces.items():
                x, y, w, h = data['box']
                emotion = data['emotion']
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                cv2.putText(frame, f"ID {profile_id}: {emotion}", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

            # Print the updated profiles and their emotions
            print("\nUpdated Profiles:")
            for pid, pdata in profiles.items():
                print(f"Profile ID {pid}: Emotions = {pdata['emotions']}")

            # Display the resulting frame
            cv2.imshow('Real-time Emotion Detection with Profiles', frame)

            # Press 'q' to exit
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        # Release the capture and close all windows
        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


2025-01-10 16:36:04.593700: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.



Updated Profiles:

Updated Profiles:

Updated Profiles:

Updated Profiles:

Updated Profiles:

Updated Profiles:

Updated Profiles:

Updated Profiles:

Updated Profiles:

Updated Profiles:

Updated Profiles:

Updated Profiles:

Updated Profiles:

Updated Profiles:

Updated Profiles:

Updated Profiles:

Updated Profiles:

Updated Profiles:

Updated Profiles:

Updated Profiles:
Profile ID 1: Emotions = ['neutral']

Updated Profiles:
Profile ID 1: Emotions = ['neutral']

Updated Profiles:
Profile ID 1: Emotions = ['neutral']

Updated Profiles:
Profile ID 1: Emotions = ['neutral']

Updated Profiles:
Profile ID 1: Emotions = ['neutral']

Updated Profiles:
Profile ID 1: Emotions = ['neutral']

Updated Profiles:
Profile ID 1: Emotions = ['neutral']

Updated Profiles:
Profile ID 1: Emotions = ['neutral']

Updated Profiles:
Profile ID 1: Emotions = ['neutral']

Updated Profiles:
Profile ID 1: Emotions = ['neutral']

Updated Profiles:
Profile ID 1: Emotions = ['neutral']

Updated Profiles:
Prof

: 

In [1]:
import cv2
from deepface import DeepFace
import numpy as np
from scipy.spatial.distance import cosine

# Configuration constants
CASCADE_PATH = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
SCALE_FACTOR = 1.1
MIN_NEIGHBORS = 5
MIN_FACE_SIZE = (30, 30)
KNOWN_FACES = {
    "Rohan": "rohan.jpg",
    "Ajay": "ajay.jpg"
}

# Load and store profiles
def load_known_faces(image_paths):
    """
    Generate embeddings for provided images.
    :param image_paths: Dictionary of {name: image_path} pairs.
    :return: Dictionary of {name: embedding} pairs.
    """
    known_faces = {}
    for name, image_path in image_paths.items():
        try:
            # Load the image
            face_image = cv2.imread(image_path)
            if face_image is None:
                print(f"Error: Could not read image {image_path}")
                continue

            # Generate embedding
            embedding = DeepFace.represent(face_image, model_name='Facenet512', enforce_detection=False)
            known_faces[name] = np.array(embedding[0]['embedding'])
            print(f"Loaded embedding for {name}")
        except Exception as e:
            print(f"Error processing {name}: {e}")
    return known_faces

# Match a face to the closest known face
def classify_face(face_embedding, known_faces):
    """
    Classify a detected face embedding to the closest known face.
    :param face_embedding: The embedding of the detected face.
    :param known_faces: Dictionary of {name: embedding} pairs.
    :return: Name of the closest face and its distance.
    """
    closest_name = None
    closest_distance = float('inf')

    for name, known_embedding in known_faces.items():
        distance = cosine(known_embedding, face_embedding)
        if distance < closest_distance:
            closest_name = name
            closest_distance = distance

    return closest_name, closest_distance

# Detect faces in a frame
def detect_faces(face_cascade, frame):
    """Detect faces in the given frame."""
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray_frame, 
        scaleFactor=SCALE_FACTOR, 
        minNeighbors=MIN_NEIGHBORS, 
        minSize=MIN_FACE_SIZE
    )
    return faces

# Generate face embedding
def get_face_embedding(face_roi):
    """Generate an embedding for the given face ROI."""
    try:
        embedding = DeepFace.represent(face_roi, model_name='Facenet512', enforce_detection=False)
        return np.array(embedding[0]['embedding'])
    except Exception as e:
        print(f"Embedding error: {e}")
        return None

# Main script
def main():
    # Load known face embeddings
    known_faces = load_known_faces(KNOWN_FACES)

    # Load face cascade classifier
    face_cascade = cv2.CascadeClassifier(CASCADE_PATH)

    # Initialize video capture
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise RuntimeError("Error: Could not open video capture.")

    try:
        while True:
            # Capture frame-by-frame
            ret, frame = cap.read()
            if not ret:
                print("Error: Unable to read frame.")
                break

            # Detect faces in the frame
            faces = detect_faces(face_cascade, frame)

            for (x, y, w, h) in faces:
                # Extract the face ROI
                face_roi = frame[y:y + h, x:x + w]
                if face_roi.size == 0:
                    continue

                # Get face embedding
                embedding = get_face_embedding(face_roi)
                if embedding is None:
                    continue

                # Classify face to the closest profile
                name, distance = classify_face(embedding, known_faces)

                # Draw rectangle and label around the face
                color = (0, 255, 0) if distance < 0.15 else (0, 255, 255)  # Green for close match, Yellow for not close
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                cv2.putText(frame, f"{name} ({distance:.2f})", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

            # Display the resulting frame
            cv2.imshow('Face Recognition (Rohan & Ajay)', frame)

            # Press 'q' to exit
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


2025-01-10 16:44:26.131761: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loaded embedding for Rohan
Loaded embedding for Ajay
